In [1]:
#pip install nltk==3.5

In [2]:
#%pip install wget

In [3]:
#%pip install py4j

In [4]:
#Install Latest Version of Spark
#%pip install pyspark

#!/usr/bin/env python3 

 

# 

# Licensed to the Apache Software Foundation (ASF) under one or more 

# contributor license agreements.  See the NOTICE file distributed with 

# this work for additional information regarding copyright ownership. 

# The ASF licenses this file to You under the Apache License, Version 2.0 

# (the "License"); you may not use this file except in compliance with 

# the License.  You may obtain a copy of the License at 

# 

#    http://www.apache.org/licenses/LICENSE-2.0 

# 

# Unless required by applicable law or agreed to in writing, software 

# distributed under the License is distributed on an "AS IS" BASIS, 

# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

# See the License for the specific language governing permissions and 

# limitations under the License. 

# 

 

# This script attempt to determine the correct setting for SPARK_HOME given 

# that Spark may have been installed on the system with pip. 

 

import os 

import sys 

 

 

def _find_spark_home(): 

    """Find the SPARK_HOME.""" 

    # If the environment has SPARK_HOME set trust it. 

    if "SPARK_HOME" in os.environ: 

        return os.environ["SPARK_HOME"] 

 

    def is_spark_home(path): 

        """Takes a path and returns true if the provided path could be a reasonable SPARK_HOME""" 

        return (os.path.isfile(os.path.join(path, "bin/spark-submit")) and 

                (os.path.isdir(os.path.join(path, "jars")) or 

                 os.path.isdir(os.path.join(path, "assembly")))) 

 

    # Spark distribution can be downloaded when PYSPARK_HADOOP_VERSION environment variable is set. 

    # We should look up this directory first, see also SPARK-32017. 

    spark_dist_dir = "spark-distribution" 

    paths = [ 

        "../",  # When we're in spark/python. 

        # Two case belows are valid when the current script is called as a library. 

        os.path.join(os.path.dirname(os.path.realpath('spark-3.3.2-bin-hadoop3.tgz')), spark_dist_dir), 

        os.path.dirname(os.path.realpath('spark-3.3.2-bin-hadoop3.tgz'))] 

 

    # Add the path of the PySpark module if it exists 

    import_error_raised = False 

    from importlib.util import find_spec 

    try: 

        module_home = os.path.dirname(find_spec("pyspark").origin) 

        paths.append(os.path.join(module_home, spark_dist_dir)) 

        paths.append(module_home) 

        # If we are installed in edit mode also look two dirs up 

        # Downloading different versions are not supported in edit mode. 

        paths.append(os.path.join(module_home, "../../")) 

    except ImportError: 

        # Not pip installed no worries 

        import_error_raised = True 

 

    # Normalize the paths 

    paths = [os.path.abspath(p) for p in paths] 

 

    try: 

        return next(path for path in paths if is_spark_home(path)) 

    except StopIteration: 

        print("Could not find valid SPARK_HOME while searching {0}".format(paths), file=sys.stderr) 

        if import_error_raised: 

            print( 

                "\nDid you install PySpark via a package manager such as pip or Conda? If so,\n" 

                "PySpark was not found in your Python environment. It is possible your\n" 

                "Python environment does not properly bind with your package manager.\n" 

                "\nPlease check your default 'python' and if you set PYSPARK_PYTHON and/or\n" 

                "PYSPARK_DRIVER_PYTHON environment variables, and see if you can import\n" 

                "PySpark, for example, 'python -c 'import pyspark'.\n" 

                "\nIf you cannot import, you can install by using the Python executable directly,\n" 

                "for example, 'python -m pip install pyspark [--user]'. Otherwise, you can also\n" 

                "explicitly set the Python executable, that has PySpark installed, to\n" 

                "PYSPARK_PYTHON or PYSPARK_DRIVER_PYTHON environment variables, for example,\n" 

                "'PYSPARK_PYTHON=python3 pyspark'.\n", file=sys.stderr) 

        sys.exit(-1) 

 

 

if __name__ == "__main__": 

    print(_find_spark_home()) 

In [5]:
#conda install -c conda-forge scitime

In [6]:
import os
import sys

In [7]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [8]:
import pandas as pd
#import wget

# Install Java, Spark, and Findspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!python -m wget https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
#!tar xf spark-3.3.2-bin-hadoop3.tgz
#!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "C:/Program Files/Java/jdk-20"
#os.environ["SPARK_HOME"] = "C:/Users/bjros/Anaconda3/Lib/site-packages/pyspark"

# Start a SparkSession
import findspark
findspark.init(r"C:\Users\bjros\Anaconda3\Lib\site-packages\pyspark")

In [9]:
os.environ["HADOOP_HOME"] = "C:/Users/bjros/OneDrive/Desktop/hadoop-3.3.5-src"

In [10]:
# Start Spark session
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [11]:
#Found method to increase Heap Size at https://stackoverflow.com/questions/32336915/pyspark-java-lang-outofmemoryerror-java-heap-space
spark = SparkSession.builder.appName("NaiveBayes").config("spark.driver.memory", "8g").config("spark.executor.memory", "8g").config("spark.driver.maxResultSize", "0").config("spark.executor.heartbeatInterval", "300s").config("spark.network.timeout", "600s").config("spark.sql.execution.arrow.pyspark.enabled", "true").getOrCreate()

In [12]:
#Found Method to Speed Up Converting Python Objects to DataFrames at spark.conf.set("spark.sql.execution.arrow.enabled", "true")
#spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [13]:
#Check Spark Version
print( "spark version=" ,SparkSession.builder.appName("test").getOrCreate().version)

spark version= 3.3.2


In [14]:
#Read the Data from the CSV Files
#complaints_df = pd.read_csv('train.csv')
#complaints_df['narrative_length'] = train_df['narrative'].str.len()

from pyspark.sql.functions import length

complaints_df_with_nulls = spark.read.csv('complaints_processed.csv', sep=",", header=True)
complaints_df_with_nulls = complaints_df_with_nulls.withColumn('narrative_length', length(complaints_df_with_nulls['narrative']))
complaints_df = complaints_df_with_nulls.dropna()
complaints_df.show(n=10)

+---+----------------+--------------------+----------------+
|_c0|         product|           narrative|narrative_length|
+---+----------------+--------------------+----------------+
|  0|     credit_card|purchase order da...|            1705|
|  1|     credit_card|forwarded message...|             904|
|  2|  retail_banking|forwarded message...|            1230|
|  3|credit_reporting|payment history m...|             903|
|  4|credit_reporting|payment history m...|             851|
|  5|credit_reporting|payment history m...|             849|
|  6|credit_reporting|va date complaint...|            1395|
|  7|credit_reporting|account reported ...|            2114|
|  8|credit_reporting|account reported ...|            2120|
|  9|credit_reporting|usdoexxxx account...|            2093|
+---+----------------+--------------------+----------------+
only showing top 10 rows



#Tokenize the Description Columns in the DataFrames
train_df['description_x_list'] = train_df['description_x'].apply(lambda x : word_tokenize(x))
train_df['description_x_list']

train_df['description_y_list'] = train_df['description_y'].apply(lambda x : word_tokenize(x))
train_df['description_y_list']

test_df['description_x_list'] = test_df['description_x'].apply(lambda x : word_tokenize(x))
test_df['description_x_list']

test_df['description_y_list'] = test_df['description_y'].apply(lambda x : word_tokenize(x))
test_df['description_y_list']

In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram, HashingTF, IDF, StringIndexer
# Create all the features to the data set
product = StringIndexer(inputCol='product',outputCol='label')

In [16]:
tokenizer = Tokenizer(inputCol="narrative", outputCol="narrative_tokenized")

In [17]:
stopremove = StopWordsRemover(inputCol='narrative_tokenized',outputCol='narrative_tokenized_filtered')

In [18]:
#Try Different NGram Values
ngram = NGram(n=1, inputCol='narrative_tokenized_filtered',outputCol='narrative_ngramed')

In [19]:
hashingTF = HashingTF(inputCol='narrative_ngramed', outputCol='narrative_hashed')

In [20]:
idf = IDF(inputCol='narrative_hashed', outputCol='narrative_idf')

In [21]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['narrative_idf', 'narrative_length'], outputCol='features')

In [22]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[product, tokenizer, stopremove, ngram, hashingTF, idf, clean_up])

#Convert Pandas DataFrames into Spark DataFrames
#Credit to https://stackoverflow.com/questions/55604506/error-attributeerror-dataframe-object-has-no-attribute-jdf
#from pyspark.sql import SQLContext

#sqlContext = SQLContext(spark)

spark_train_df = sqlContext.createDataFrame(train_df)
spark_test_df = sqlContext.createDataFrame(test_df)

#spark_train_df.show()

#spark_test_df.show()

In [23]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(complaints_df)

cleaned = cleaner.transform(complaints_df)

In [24]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  3.0|(262145,[511,991,...|
|  3.0|(262145,[4075,552...|
|  4.0|(262145,[3436,538...|
|  0.0|(262145,[6122,853...|
|  0.0|(262145,[6122,853...|
|  0.0|(262145,[6122,853...|
|  0.0|(262145,[2306,688...|
|  0.0|(262145,[2306,421...|
|  0.0|(262145,[2306,421...|
|  0.0|(262145,[2306,421...|
|  2.0|(262145,[2162,407...|
|  3.0|(262145,[651,1139...|
|  3.0|(262145,[991,2182...|
|  0.0|(262145,[25363,48...|
|  1.0|(262145,[5381,612...|
|  2.0|(262145,[1968,164...|
|  2.0|(262145,[11395,16...|
|  2.0|(262145,[3928,507...|
|  2.0|(262145,[6122,113...|
|  0.0|(262145,[4214,211...|
+-----+--------------------+
only showing top 20 rows



In [25]:
#pip install pyspark[sql]
#spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [26]:
#Try a Naive Bayes Model for Classification
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
complaints_df_train, complaints_df_test = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(complaints_df_train)

#Convert PySpark DataFrame(s) to Pandas DataFrame(s)
cleaned_Pandas = cleaned.toPandas()

#Method Found at https://sparkbyexamples.com/pyspark/pyspark-collect/
#cleaned_X_Pandas = pd.DataFrame.from_records(cleaned_X.collect(), columns=cleaned_X.columns)
#cleaned_Y_Pandas = pd.DataFrame.from_records(cleaned_Y.collect(), columns=cleaned_Y.columns)

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

#Try a Logistic Regression Model
from pyspark.ml.classification import LogisticRegression

# Break data down into a training set and a testing set
train_df_train_X, train_df_test_X = cleaned_X.randomSplit([0.7, 0.3])
train_df_train_Y, train_df_test_Y = cleaned_Y.randomSplit([0.7, 0.3])

# Create a Logistic Regression model and fit training data
lr = LogisticRegression()
predictor_X = lr.fit(train_df_train_X)
predictor_Y = lr.fit(train_df_train_Y)

#Try a Decision Tree Model
from sklearn.tree import DecisionTreeClassifier

# dividing X, y into train and test data
complaints_df_train_features, complaints_df_test_features, complaints_df_train_labels, complaints_df_test_labels = train_test_split(cleaned_Pandas['features'], cleaned_Pandas['label'], random_state = 42)                                                                                                                                      
  

  
# creating a confusion matrix
#cm_X = confusion_matrix(train_df_test_X_labels, dtree_predictions_X)
#cm_Y = confusion_matrix(train_df_test_Y_labels, dtree_predictions_Y)

#Create the Models
dtree_model = DecisionTreeClassifier(max_depth = 2)

In [27]:
#pip install -U pip

pip install --install-option='--prefix=C:\Users\bjros\Anaconda3\Lib\site-packages' scitime

conda install -c conda-forge scitime

#Check to See how long the Model Would Take to Fit
import time

from scitime import RuntimeEstimator
estimator = RuntimeEstimator(meta_algo='RF', verbose=3)

estimation, lower_bound, upper_bound = estimator.time(dtree_model, list(complaints_df_train_features))
print(f"The time estimation for fitting dtree_model is: {estimation}. The lower bound is: {lower_bound}. The upper bound is: {upper_bound}.")

# training a DescisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtree_model_trained = dtree_model.fit(list(complaints_df_train_features), complaints_df_train_labels)

dtree_predictions = dtree_model.predict(list(complaints_df_test_features))

# Create a Naive Bayes model with Bernoulli Model Type and fit training data
nb_gaussian = NaiveBayes(modelType='gaussian')
predictor_X_gaussian = nb_gaussian.fit(train_df_train_X)
predictor_Y_gaussian = nb_gaussian.fit(train_df_train_Y)

In [28]:
# Tranform the model with the testing data
test_results = predictor.transform(complaints_df_test)
test_results.show(5)

+-----+-------------------+--------------------+----------------+-----+--------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  _c0|            product|           narrative|narrative_length|label| narrative_tokenized|narrative_tokenized_filtered|   narrative_ngramed|    narrative_hashed|       narrative_idf|            features|       rawPrediction|         probability|prediction|
+-----+-------------------+--------------------+----------------+-----+--------------------+----------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  100|    debt_collection|wakefield associa...|             588|  1.0|[wakefield, assoc...|        [wakefield, assoc...|[wakefield, assoc...|(262144,[5078,612...|(262144,[5078,612...|(262145,[5078,612...|[-2762.6973737426.

# Tranform the Bernoulli Type model with the testing data
test_results_X_gaussian = predictor_X_gaussian.transform(train_df_test_X)
test_results_X_gaussian.show(5)

test_results_Y_gaussian = predictor_Y_gaussian.transform(train_df_test_Y)
test_results_Y_gaussian.show(5)

In [29]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Test Accuracy of Base Model
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting product class values was: %f" % acc)

Accuracy of model at predicting product class values was: 0.814481


# Test Accuracy of Model with Gaussian Type
acc_eval_gaussian = MulticlassClassificationEvaluator()
acc_X_gaussian = acc_eval_gaussian.evaluate(test_results_X)
print("Accuracy of Gaussian Type model at predicting Ticker_X values was: %f" % acc_X_gaussian)

acc_eval_gaussian = MulticlassClassificationEvaluator()
acc_Y_gaussian = acc_eval_gaussian.evaluate(test_results_Y)
print("Accuracy of Gaussian Type model at predicting Ticker_Y values was: %f" % acc_Y_gaussian)

# Use the Classification Report to Evaluate the Accuracy of the Decision Tree Model Predictions
from sklearn.metrics import classification_report

print(f'The results of the Decision Tree model in predicting tickers is: {classification_report(complaints_df_test_labels, dtree_predictions)}')

In [30]:
#http://localhost:4040/api/v1/applications/[base-app-id]/logs

In [31]:
#Write to CSV File
#Found Method to Concetanate Strings in PySpark with https://stackoverflow.com/questions/40426106/spark-2-0-x-dump-a-csv-file-from-a-dataframe-containing-one-array-of-type-string
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string, StringType())

test_results_1 = test_results.withColumn('narrative_length_as_str', test_results["narrative_length"].cast("string")) \
    .withColumn('label_as_str', test_results["label"].cast("string")) \
    .withColumn('narrative_tokenized_as_str', array_to_string_udf(test_results["narrative_tokenized"])) \
    .withColumn('narrative_tokenized_filtered_as_str', array_to_string_udf(test_results["narrative_tokenized_filtered"])) \
    .withColumn('narrative_ngramed_as_str', array_to_string_udf(test_results["narrative_ngramed"])) \
    .withColumn('narrative_hashed_as_str', array_to_string_udf(test_results["narrative_hashed"])) \
    .withColumn('narrative_idf_as_str', array_to_string_udf(test_results["narrative_idf"])) \
    .withColumn('features_as_str', array_to_string_udf(test_results["features"])) \
    .withColumn('rawPrediction_as_str', array_to_string_udf(test_results["rawPrediction"])) \
    .withColumn('probability_as_str', array_to_string_udf(test_results["probability"]))

test_results_2 = test_results_1.drop("narrative_length") \
    .drop("label") \
    .drop("narrative_tokenized") \
    .drop("narrative_tokenized_filtered") \
    .drop("narrative_ngramed") \
    .drop("narrative_hashed") \
    .drop("narrative_idf") \
    .drop("features") \
    .drop("rawPrediction") \
    .drop("probability")

test_results_2.show()

+-----+-------------------+--------------------+----------+-----------------------+------------+--------------------------+-----------------------------------+------------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+
|  _c0|            product|           narrative|prediction|narrative_length_as_str|label_as_str|narrative_tokenized_as_str|narrative_tokenized_filtered_as_str|narrative_ngramed_as_str|narrative_hashed_as_str|narrative_idf_as_str|     features_as_str|rawPrediction_as_str|  probability_as_str|
+-----+-------------------+--------------------+----------+-----------------------+------------+--------------------------+-----------------------------------+------------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+
|  100|    debt_collection|wakefield associa...|       1.0|                    588|         1.0|      [wakefield,associ..

#Set Socket Wait Time
import socket
socket.setdefaulttimeout(10)
sock = socket.socket()
sock.settimeout(10)

In [32]:
#Write to a CSV File
#test_results_2.select(["_c0", "product", "narrative"]).write.csv('C:/Users/bjros/OneDrive/Desktop/Kaggle Data Sets/Consumer_Complaints/complaints_with_predictions.csv')

In [38]:
#Write Results to a File
#test_results_2.write.csv('C:/Users/bjros/OneDrive/Desktop/Kaggle Data Sets/Consumer_Complaints/complaints_with_predictions.csv')
test_results_2.select(["_c0", "product", "narrative", "label_as_str", "prediction"]).toPandas().to_csv('C:/Users/bjros/OneDrive/Desktop/Kaggle Data Sets/Consumer_Complaints/complaints_with_predictions.csv')

In [ ]:
#Results
#Multi-class Classification Evaluator
#N Grams = 1
#acc = [0.810295, 0.811987, 0.812360]
#acc_gaussian = 

#N Grams = 2
#acc = [0.797944, 0.800598, 0.797773]
#acc_gaussian = 


#N Grams = 3
#acc = [0.619139, 0.615107, 0.615909]
#acc_gaussian = 


#Logistic Regression
#acc_X =
#acc_Y = 